In [1]:
# Note: 1. Always double check your path, data name, and extension
# 2. Close the file to avoid conflicts
# 3. Use field view to check actual column names and alias

In [1]:
# Now let's try to load the basic data and explore them
# As usual, you need to set up your Arcpy
import arcpy

arcpy.env.workspace = r'C:\Users\nismayil\OneDrive - Texas Tech University\GeoStatistics\Lab 12'
arcpy.env.overwriteOutput = True

# Define the Nighttime Light file path - note I have renamed this to DMSP NTL
dmsp_path = 'dmsp.tif'

# Describe the GeoTIFF to get its properties
desc = arcpy.Describe(dmsp_path)

# Print basic properties
print("Data Type ",desc.dataType)
print("Spatial Reference ", desc.spatialReference.name)
print("Extent ", desc.extent)
print("Number of Bands ",desc.bandCount)

Data Type  RasterDataset
Spatial Reference  GCS_WGS_1984
Extent  -180.00416666665 -65.00416610665 180.00416522665 75.00416666665 NaN NaN NaN NaN
Number of Bands  1


In [2]:
# Explore the admin boundaray shapefile
shapefile_path = r'C:\Users\nismayil\OneDrive - Texas Tech University\GeoStatistics\Lab 12\ADM0\geoBoundariesCGAZ_ADM0.shp' # define your path here

# List all fields in the shapefile
fields = arcpy.ListFields(shapefile_path)

# Print column names
print("Columns in the admin shapefile:")
for field in fields:
    print(field.name)

Columns in the admin shapefile:
FID
Shape
shapeGroup
shapeType
shapeName


In [3]:
# use search cursor to preview the value

# Create a search cursor to iterate over rows
with arcpy.da.SearchCursor(shapefile_path, ["shapeGroup", "shapeType", "shapeName"]) as cursor: 
    for row in cursor:
        # Print each row
        print(row[0], row[1], row[2])

AFG ADM0 Afghanistan
GBR ADM0 United Kingdom
ALB ADM0 Albania
DZA ADM0 Algeria
USA ADM0 United States
ATA ADM0 Antarctica
ATG ADM0 Antigua & Barbuda
ARG ADM0 Argentina
AND ADM0 Andorra
AGO ADM0 Angola
ARM ADM0 Armenia
NLD ADM0 Netherlands
AUS ADM0 Australia
AUT ADM0 Austria
AZE ADM0 Azerbaijan
BHS ADM0 Bahamas, The
BRB ADM0 Barbados
BLR ADM0 Belarus
BEL ADM0 Belgium
BLZ ADM0 Belize
BTN ADM0 Bhutan
BOL ADM0 Bolivia
BIH ADM0 Bosnia & Herzegovina
BWA ADM0 Botswana
NOR ADM0 Norway
COL ADM0 Colombia
COM ADM0 Comoros
BRA ADM0 Brazil
FRA ADM0 France
COD ADM0 Congo, Dem Rep of the
COG ADM0 Congo, Rep of the
NZL ADM0 New Zealand
BRN ADM0 Brunei
BGR ADM0 Bulgaria
BFA ADM0 Burkina Faso
BDI ADM0 Burundi
CPV ADM0 Cabo Verde
KHM ADM0 Cambodia
CMR ADM0 Cameroon
CAN ADM0 Canada
CRI ADM0 Costa Rica
CIV ADM0 Cote d'Ivoire
HRV ADM0 Croatia
CUB ADM0 Cuba
CAF ADM0 Central African Rep
TCD ADM0 Chad
CHL ADM0 Chile
CHN ADM0 China
CYP ADM0 Cyprus
CZE ADM0 Czechia
DNK ADM0 Denmark
DJI ADM0 Djibouti
DMA ADM0 Dom

In [4]:
# Explore the population table with Pandas
# more about pandas: https://pandas.pydata.org/
import pandas as pd
path_pop = r'C:\Users\nismayil\OneDrive - Texas Tech University\GeoStatistics\Lab 12\pop2013.csv'
pop = pd.read_csv(path_pop)
pop.head()
# you may notice that "shapeGroup" and "Country Code" contains same country code

,Country Name,Country Code,2013POP
0,Aruba,ABW,102880.0
1,Africa Eastern and Southern,AFE,567892149.0
2,Afghanistan,AFG,31541209.0
3,Africa Western and Central,AFW,387204553.0
4,Angola,AGO,26147002.0


In [6]:
# Create a geodatabase to save all these data in the gdb
geodatabase_name = 'lab.gdb'
arcpy.CreateFileGDB_management(arcpy.env.workspace, geodatabase_name)

<Result 'C:\\Users\\nismayil\\OneDrive - Texas Tech University\\GeoStatistics\\Lab 12\\lab.gdb'>

In [7]:
# save shapefile to gdp
arcpy.FeatureClassToGeodatabase_conversion(shapefile_path, geodatabase_name)

<Result 'C:\\Users\\nismayil\\OneDrive - Texas Tech University\\GeoStatistics\\Lab 12\\lab.gdb'>

In [8]:
# save gdp table
# Name of the output table in the geodatabase
output_table_name = 'pop'  
arcpy.TableToTable_conversion(path_pop, geodatabase_name, output_table_name)

<Result 'C:\\Users\\nismayil\\OneDrive - Texas Tech University\\GeoStatistics\\Lab 12\\lab.gdb\\pop'>

In [9]:
# now change the workspace to gdb
arcpy.env.workspace = r'C:\Users\nismayil\OneDrive - Texas Tech University\GeoStatistics\Lab 12\lab.gdb'

In [10]:
# save dmsp to gdp
dmsp_paths= r'C:\Users\nismayil\OneDrive - Texas Tech University\GeoStatistics\Lab 12\dmsp.tif'
arcpy.management.CopyRaster(dmsp_paths, 'dmsp')

<Result 'C:\\Users\\nismayil\\OneDrive - Texas Tech University\\GeoStatistics\\Lab 12\\lab.gdb\\dmsp'>

In [12]:
# Open ArcGIS and check to gdb
arcpy.FeatureClassToGeodatabase_conversion(shapefile_path, geodatabase_name)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Output Geodatabase: Dataset lab.gdb does not exist or is not supported
Failed to execute (FeatureClassToGeodatabase).


In [12]:
# Combine population data with the admin shapefile

# Field to join on in the shapefile
join_field1 = 'shapeGroup'

# Field to join on in the table
join_field2 = 'Country_Code'

# Output feature class name after the join
output_feature_class = 'admin_pop'

# Join the table to the shapefile
joined_table = arcpy.management.AddJoin("geoBoundariesCGAZ_ADM0", join_field1, "pop", join_field2)

# See field names and aliases
resultFields = arcpy.ListFields(joined_table)
print([field.name for field in resultFields])
      
# Copy the result to a new feature class
arcpy.management.CopyFeatures(joined_table, output_feature_class) # you can check your result

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Layer Name or Table View: Dataset geoBoundariesCGAZ_ADM0 does not exist or is not supported
Failed to execute (AddJoin).


In [28]:
# use zonal statistics to calculate the total nighttime light intensity in the country
# source: https://pro.arcgis.com/en/pro-app/latest/tool-reference/spatial-analyst/zonal-statistics-as-table.htm
from arcpy.sa import *
outZSaT = ZonalStatisticsAsTable("geoBoundariesCGAZ_ADM0", "shapeGroup", "dmsp","country_pop", "NODATA", "SUM")

In [29]:
# you can check your pop table in arcgis
# Combine total ntl data with the admin shapefile

# Field to join on in the shapefile
join_field1 = 'geoBoundariesCGAZ_ADM0_shapeGroup'

# Field to join on in the table
join_field2 = 'shapeGroup'

# Output feature class name after the join
output_feature_class = 'admin_pop_ntl'

# Join the table to the shapefile
joined_table = arcpy.management.AddJoin("admin_pop", join_field1, "country_pop", join_field2)

# See field names and aliases
resultFields = arcpy.ListFields(joined_table)
print([field.name for field in resultFields])
      
# Copy the result to a new feature class
arcpy.management.CopyFeatures(joined_table, output_feature_class) # you can check your result

['admin_pop.OBJECTID', 'admin_pop.Shape', 'admin_pop.geoBoundariesCGAZ_ADM0_shapeGroup', 'admin_pop.geoBoundariesCGAZ_ADM0_shapeType', 'admin_pop.geoBoundariesCGAZ_ADM0_shapeName', 'admin_pop.pop_OBJECTID', 'admin_pop.pop_Country_Name', 'admin_pop.pop_Country_Code', 'admin_pop.pop_F2013POP', 'admin_pop.Shape_Length', 'admin_pop.Shape_Area', 'country_pop.OBJECTID', 'country_pop.shapeGroup', 'country_pop.ZONE_CODE', 'country_pop.COUNT', 'country_pop.AREA', 'country_pop.SUM']


<Result 'C:\\Users\\nismayil\\OneDrive - Texas Tech University\\GeoStatistics\\Lab 12\\lab12.gdb\\admin_pop_ntl'>

In [34]:
# Sum is the lighttime light intensity
# Now let's plot
import matplotlib.pyplot as plt
import numpy as np

# Names of the columns to use for plotting
x_column_name = 'admin_pop_pop_F2013POP'
y_column_name = 'country_pop_SUM'

# Retrieve values from the columns using a cursor
x_values = []
y_values = []

with arcpy.da.SearchCursor('admin_pop_ntl', [x_column_name, y_column_name]) as cursor:
    for row in cursor:
        x_values.append(row[0])
        y_values.append(row[1])

# Plot the scatter plot
plt.scatter(x_values, y_values)


# Add labels and legend
plt.xlabel(x_column_name)
plt.ylabel(y_column_name)
plt.legend()

# Show the plot
plt.show()


No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


In [37]:
# try to plot in ArcGIS
# What is the R2
# Select two income groups and compare your results
# https://datahelpdesk.worldbank.org/knowledgebase/articles/906519-world-bank-country-and-lending-groups
# You can use either arcgis or python to select your data
# How would the calculation of R-squared differ when segmented by income groups? 
# What insights can we derive from analyzing R-squared within distinct income brackets?


#1


import pandas as pd
path_class = r'C:\Users\nismayil\OneDrive - Texas Tech University\GeoStatistics\Lab 12\CLASS.csv'
class1 = pd.read_csv(path_class)
class1.head()





,Economy,Code,Region,Income group,Lending category,Unnamed: 5
0,Aruba,ABW,Latin America & Caribbean,High income,NaN,NaN
1,Afghanistan,AFG,South Asia,Low income,IDA,NaN
2,Angola,AGO,Sub-Saharan Africa,Lower middle income,IBRD,NaN
3,Albania,ALB,Europe & Central Asia,Upper middle income,IBRD,NaN
4,Andorra,AND,Europe & Central Asia,High income,NaN,NaN


In [38]:
arcpy.FeatureClassToGeodatabase_conversion(path_class, geodatabase_name)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Input Features: Dataset C:\Users\nismayil\OneDrive - Texas Tech University\GeoStatistics\Lab 12\CLASS.csv does not exist or is not supported
ERROR 000732: Output Geodatabase: Dataset lab12.gdb does not exist or is not supported
Failed to execute (FeatureClassToGeodatabase).
